In [1]:
import numpy as np
import pandas as pd

from config import BASE_DIR
from notebooks.notebooks_utils import set_seed_globaly


RERUN = False

In [2]:


clustering_label = "composition"
CHUNK_SIZE = 100
DATA_PATH = f"{BASE_DIR}/notebooks/data/filtered_text_and_starr_features_{CHUNK_SIZE}_words_nonbib_17_06_2024.csv"
df = pd.read_csv(DATA_PATH)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1365 entries, 0 to 1364
Data columns (total 25 columns):
 #   Column                                                              Non-Null Count  Dtype  
---  ------                                                              --------------  -----  
 0   book                                                                1365 non-null   object 
 1   sentence_path                                                       1365 non-null   object 
 2   text_lex                                                            1362 non-null   object 
 3   text                                                                1365 non-null   object 
 4   n_words_lex                                                         1365 non-null   int64  
 5   n_words                                                             1365 non-null   int64  
 6   construct to absolute nouns ratio                                   1365 non-null   float64
 7   construct nouns

In [3]:

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from notebooks.clustering_main import stratified_sample
set_seed_globaly()
idx_to_remove_composition = df['composition'].isna()
df_compoisition_classification = df[~idx_to_remove_composition]
# df_compoisition_classification = stratified_sample(df_compoisition_classification, 'composition', fr)
# comp_counts = df_compoisition_classification["composition"].value_counts()
# relevant_comps = comp_counts[comp_counts>5]
# df_compoisition_classification[df_compoisition_classification["composition"].isin(relevant_comps)]

train_idx, test_idx = train_test_split(df_compoisition_classification.index, test_size=0.3, random_state=42, stratify=df_compoisition_classification['composition'])
df_compoisition_classification.loc[train_idx, 'split'] = 'train'
df_compoisition_classification.loc[test_idx, 'split'] = 'test'
df_compoisition_classification["split"].value_counts()


/var/folders/yv/8bscwg3x6m5934xb_mry4d_m0000gr/T/ipykernel_28258/1318954615.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_compoisition_classification.loc[train_idx, 'split'] = 'train'
/var/folders/yv/8bscwg3x6m5934xb_mry4d_m0000gr/T/ipykernel_28258/1318954615.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'train' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_compoisition_classification.loc[train_idx, 'split'] = 'train'


split
train    568
test     244
Name: count, dtype: int64

In [4]:
df_compoisition_classification.to_csv(f"{BASE_DIR}/notebooks/data/df_compoisition_classification.csv", index=False)

In [5]:
PATH_TO_FILE = "/Users/yonatanlou/dev/BertGCN/data"
df_compoisition_classification[["sentence_path", "split","composition"]].to_csv(f"{PATH_TO_FILE}/DSS_composition_classification_labels.txt", index=False, header=False, sep="\t")
df_compoisition_classification[["text"]].to_csv(f"{PATH_TO_FILE}/corpus/DSS_composition_classification_text.txt", index=False, header=False, sep="\t")
# idx_to_remove_sectarian = (df['section'].isna()) | (df['section'] == "unknown")
# df_sectarian_classification = df[~idx_to_remove_sectarian]

In [18]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
def compute_classification_metrics(model, vectorizer_matrix, df, train_idx, test_idx, label_column):
    
    le = LabelEncoder()
    df['encoded_labels'] = le.fit_transform(df[label_column])

    X_train, y_train = vectorizer_matrix[train_idx],df.loc[train_idx,'encoded_labels']
    X_test, y_test = vectorizer_matrix[test_idx],df.loc[test_idx,'encoded_labels'] 
    # Fit the model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Compute evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    metrics = {
        'model': type(model).__name__,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

    return metrics


In [6]:
df_compoisition_classification =  pd.read_csv(f"{PATH_TO_FILE}/DSS_composition_classification_labels.txt", sep="\t", names=["sentence_path", "split","composition"])


In [11]:
from notebooks.features import vectorize_text

X = vectorize_text(df, "text", "dicta-il/BEREL")

/Users/yonatanlou/.pyenv/versions/3.10.0/envs/QumranNLP/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

KeyboardInterrupt



In [22]:
# Define the models and vectorizers to be evaluated
models = [
    RandomForestClassifier(random_state=42),
    LogisticRegression(max_iter=200),
    SVC(),
    KNeighborsClassifier(),
    AdaBoostClassifier(random_state=42),
]
metrics_list = []
label_column="composition"
train_idx, test_idx = df_compoisition_classification[df_compoisition_classification['split']=="train"].index, df_compoisition_classification[df_compoisition_classification['split']=="test"].index
# Iterate over each model and vectorizer, compute metrics
for model in tqdm(models, desc="model"):
    metrics = compute_classification_metrics(model, X, df_compoisition_classification.reset_index(), train_idx, test_idx, label_column)
    metrics_list.append(metrics)

# Convert metrics list to a DataFrame for easier analysis
metrics_df = pd.DataFrame(metrics_list)
metrics_df

model:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/yonatanlou/.pyenv/versions/3.10.0/envs/QumranNLP/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yonatanlou/.pyenv/versions/3.10.0/envs/QumranNLP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/yonatanlou/.pyenv/versions/3.10.0/envs/QumranNLP/lib/python3.10/site-packages/sklearn/metrics

,model,accuracy,precision,recall,f1_score
0,RandomForestClassifier,0.429752,0.384114,0.429752,0.376516
1,LogisticRegression,0.537190,0.516249,0.537190,0.509444
2,SVC,0.297521,0.145616,0.297521,0.181927
3,KNeighborsClassifier,0.479339,0.520041,0.479339,0.478244
4,AdaBoostClassifier,0.132231,0.079574,0.132231,0.076868


In [49]:
X_tfidf = vectorize_text(df, "text", "tfidf")
# combined_X = np.hstack((X, X_starr))
# combined_X = np.hstack((X_starr,X))
# Repeat X_starr elements across the columns of X
X_bert_expanded = np.repeat(X, X_tfidf.shape[1] // X.shape[0], axis=1)

combined_X = X * X_bert_expanded
# combined_X

array([[ 0.02792844,  0.02792844,  0.02792844, ..., -0.06268632,
        -0.06268632, -0.06268632],
       [ 0.15373543,  0.15373543,  0.15373543, ..., -0.08678609,
        -0.08678609, -0.08678609],
       [ 0.16478008,  0.16478008,  0.16478008, ..., -0.10580266,
        -0.10580266, -0.10580266],
       ...,
       [ 0.11626823,  0.11626823,  0.11626823, ..., -0.01143604,
        -0.01143604, -0.01143604],
       [ 0.01994579,  0.01994579,  0.01994579, ..., -0.17171119,
        -0.17171119, -0.17171119],
       [ 0.31367338,  0.31367338,  0.31367338, ..., -0.18568449,
        -0.18568449, -0.18568449]], dtype=float32)

In [37]:
models = [
    RandomForestClassifier(random_state=42),
    LogisticRegression(max_iter=200),
    SVC(),
    KNeighborsClassifier(),
    AdaBoostClassifier(random_state=42),
]
metrics_list = []
label_column="composition"
train_idx, test_idx = df_compoisition_classification[df_compoisition_classification['split']=="train"].index, df_compoisition_classification[df_compoisition_classification['split']=="test"].index
# Iterate over each model and vectorizer, compute metrics
for model in tqdm(models, desc="model"):
    metrics = compute_classification_metrics(model, combined_X, df_compoisition_classification.reset_index(), train_idx, test_idx, label_column)
    metrics_list.append(metrics)

# Convert metrics list to a DataFrame for easier analysis
metrics_df = pd.DataFrame(metrics_list)
metrics_df

model:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/yonatanlou/.pyenv/versions/3.10.0/envs/QumranNLP/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yonatanlou/.pyenv/versions/3.10.0/envs/QumranNLP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/yonatanlou/.pyenv/versions/3.10.0/envs/QumranNLP/lib/python3.10/site-packages/sklearn/metrics

,model,accuracy,precision,recall,f1_score
0,RandomForestClassifier,0.413223,0.321741,0.413223,0.344265
1,LogisticRegression,0.438017,0.401141,0.438017,0.414078
2,SVC,0.148760,0.041274,0.148760,0.049599
3,KNeighborsClassifier,0.330579,0.316234,0.330579,0.313565
4,AdaBoostClassifier,0.132231,0.079574,0.132231,0.076868
